### Neural Machine Translation using word level language model and embeddings in Keras

In [175]:
#https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7

import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.model_selection import train_test_split
# Building a english to french translator

In [176]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [177]:
#lines= pd.read_table('fra.txt', names=['eng', 'fr'], index_col=False)
# read files and create dataframe
toxicity_comments = pd.read_csv('toxicity_annotated_comments.tsv', sep = '\t', index_col = 0)
toxicity_annotations = pd.read_csv('toxicity_annotations.tsv',  sep = '\t')
toxicity_demographics = pd.read_csv('toxicity_worker_demographics.tsv', sep = '\t')

toxicity = toxicity_comments.merge(toxicity_annotations, how ='outer', on="rev_id")
toxicity = toxicity.merge(toxicity_demographics, how ='outer', on="worker_id").sort_values(by=['rev_id','worker_id'])

# remove newline and tab tokens
toxicity['comment'] = toxicity['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
toxicity['comment'] = toxicity['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

# add binary gender columns
toxicity = toxicity[toxicity['gender']!='other']
toxicity = pd.concat([toxicity, pd.get_dummies(toxicity.gender).rename(columns = "{}_binary".format)], axis = 1)

# limit size of dataset for testing purposes
very_toxic = toxicity[toxicity.toxicity_score == -2.0]
lines = very_toxic[very_toxic.female_binary == 1]

lines['comment'] = lines['comment'].str.translate(str.maketrans('', '', string.punctuation))
lines['comment'] = lines['comment'].replace('\n',' ', regex=True)
lines['comment'] = lines['comment'].str.lower()
# female_vtoxic['comment'] = female_vtoxic['comment'].str.split() 
display(lines.head(5))

# comments = female_vtoxic.comment.values

/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,rev_id,comment,year,logged_in,ns,sample,split,worker_id,toxicity,toxicity_score,gender,english_first_language,age_group,education,female_binary,male_binary
188608,285223.0,oh never mind i misread the change my mist...,2001,True,article,random,train,3982,1,-2.0,female,0.0,18-30,bachelors,1,0
676542,1266286.0,erik for crying out loud you legally can h...,2003,True,user,random,test,1727,1,-2.0,female,0.0,30-45,hs,1,0
188610,1735419.0,pfortuny on pfortunys mediation apologies ...,2003,True,article,random,train,3982,1,-2.0,female,0.0,18-30,bachelors,1,0
860295,2878618.0,the arbitration committee banned plautus sa...,2004,True,user,random,train,2727,1,-2.0,female,0.0,18-30,some,1,0
1180799,4476405.0,wow youre so clever so smooth stop being an a...,2004,False,article,random,train,4096,1,-2.0,female,1.0,30-45,bachelors,1,0


In [178]:
#lines = lines[0:50000]

In [179]:
lines.sample(10)

,rev_id,comment,year,logged_in,ns,sample,split,worker_id,toxicity,toxicity_score,gender,english_first_language,age_group,education,female_binary,male_binary
344980,524883506.0,umm editor nyttend you posted the following at...,2012,True,user,random,train,1314,1,-2.0,female,0.0,30-45,masters,1,0
1100426,680945141.0,your moms yummy cum fucking bastard you s...,2015,False,user,blocked,train,1268,1,-2.0,female,0.0,18-30,bachelors,1,0
356695,420900758.0,im going to punch jim wales one day ill find...,2011,False,user,blocked,train,2235,1,-2.0,female,0.0,30-45,hs,1,0
950300,84539510.0,ask your mother miss or sir and stop talking ...,2006,True,article,blocked,train,3372,1,-2.0,female,0.0,18-30,bachelors,1,0
372345,327697645.0,it would probably be helpful if you could find...,2009,True,user,random,test,1101,1,-2.0,female,0.0,18-30,hs,1,0
576055,60493059.0,youre full of shit,2006,True,user,blocked,train,1598,1,-2.0,female,0.0,45-60,bachelors,1,0
470869,104976139.0,with regards to your comments on back the f...,2007,True,user,blocked,test,3998,1,-2.0,female,0.0,18-30,bachelors,1,0
525890,198536345.0,such distortion calls for dispute resolution ...,2008,True,user,random,train,4105,1,-2.0,female,0.0,18-30,bachelors,1,0
64237,203234647.0,racist now you know who you are and where...,2008,False,user,blocked,dev,1062,1,-2.0,female,1.0,30-45,masters,1,0
1538950,133316080.0,dont argue with that farrakhanistic moron ...,2007,True,article,blocked,train,2868,1,-2.0,female,0.0,45-60,hs,1,0


#### Data Cleanup

In [180]:
#lines.eng=lines.eng.apply(lambda x: x.lower())
#lines.fr=lines.eng.apply(lambda x: x.lower())

In [181]:
# Take the length as 50
lines['eng']=lines.comment.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines['fr']=lines.comment.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [182]:
# exclude = set(string.punctuation)
# lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [183]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.fr=lines.fr.apply(lambda x: x.translate(remove_digits))

/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [184]:
lines.head()

,rev_id,comment,year,logged_in,ns,sample,split,worker_id,toxicity,toxicity_score,gender,english_first_language,age_group,education,female_binary,male_binary,eng,fr
188608,285223.0,oh never mind i misread the change my mist...,2001,True,article,random,train,3982,1,-2.0,female,0.0,18-30,bachelors,1,0,oh never mind i misread the change my mist...,oh never mind i misread the change my mist...
676542,1266286.0,erik for crying out loud you legally can h...,2003,True,user,random,test,1727,1,-2.0,female,0.0,30-45,hs,1,0,erik for crying out loud you legally can h...,erik for crying out loud you legally can h...
188610,1735419.0,pfortuny on pfortunys mediation apologies ...,2003,True,article,random,train,3982,1,-2.0,female,0.0,18-30,bachelors,1,0,pfortuny on pfortunys mediation apologies ...,pfortuny on pfortunys mediation apologies ...
860295,2878618.0,the arbitration committee banned plautus sa...,2004,True,user,random,train,2727,1,-2.0,female,0.0,18-30,some,1,0,the arbitration committee banned plautus sa...,the arbitration committee banned plautus sa...
1180799,4476405.0,wow youre so clever so smooth stop being an a...,2004,False,article,random,train,4096,1,-2.0,female,1.0,30-45,bachelors,1,0,wow youre so clever so smooth stop being an a...,wow youre so clever so smooth stop being an a...


#### Generate synthetic data

In [185]:
lines.fr = lines.fr.apply(lambda x : 'START_ '+ x + ' _END')

/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [186]:
all_eng_words=set()
counter = {}
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
            counter[word] = 1
        else:
            counter[word]+=1
print(len(all_eng_words))
for key in counter.keys():
    if counter[key] == 1:
        all_eng_words.remove(key)
print(len(all_eng_words))
    
all_french_words=set()
dict_fr = {}
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)
            dict_fr[word] = 1
        else:
            dict_fr[word]+=1
print(len(all_french_words))
for key in dict_fr.keys():
    if dict_fr[key] == 1:
        all_french_words.remove(key)
print(len(all_french_words))

31802
18392
31804
18394


In [187]:
len(all_eng_words), len(all_french_words)

(18392, 18394)

In [188]:
clean_list=[]
for l in lines.fr:
    line = l.split(' ')
    new_line = []
    for word in line:
        if word in all_french_words:
            new_line.append(word)
    clean_list.append(new_line)
lines['clean'] = clean_list
lines['length'] = lines.apply (lambda row: len(row.clean), axis=1)
lines =lines[lines['length']<60]
display(lines.head(5))
print(lines['length'].max())

/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,rev_id,comment,year,logged_in,ns,sample,split,worker_id,toxicity,toxicity_score,gender,english_first_language,age_group,education,female_binary,male_binary,eng,fr,clean,length
188608,285223.0,oh never mind i misread the change my mist...,2001,True,article,random,train,3982,1,-2.0,female,0.0,18-30,bachelors,1,0,oh never mind i misread the change my mist...,START_ oh never mind i misread the change ...,"[START_, oh, never, mind, i, misread, the, cha...",11
860295,2878618.0,the arbitration committee banned plautus sa...,2004,True,user,random,train,2727,1,-2.0,female,0.0,18-30,some,1,0,the arbitration committee banned plautus sa...,START_ the arbitration committee banned pla...,"[START_, the, arbitration, committee, banned, ...",29
1180799,4476405.0,wow youre so clever so smooth stop being an a...,2004,False,article,random,train,4096,1,-2.0,female,1.0,30-45,bachelors,1,0,wow youre so clever so smooth stop being an a...,START_ wow youre so clever so smooth stop bei...,"[START_, wow, youre, so, clever, so, smooth, s...",16
704969,4632658.0,i have a dick its bigger than yours hahaha,2004,False,article,blocked,train,459,1,-2.0,female,0.0,45-60,masters,1,0,i have a dick its bigger than yours hahaha,START_ i have a dick its bigger than yours hah...,"[START_, i, have, a, dick, its, bigger, than, ...",11
1313372,4788677.0,lowlevel languages such as machine code are ...,2004,False,article,random,dev,3037,1,-2.0,female,0.0,18-30,bachelors,1,0,lowlevel languages such as machine code are ...,START_ lowlevel languages such as machine co...,"[START_, languages, such, as, machine, code, a...",25


59


In [189]:
lines['fr'] = lines.apply (lambda row: ' '.join(row.clean), axis=1)
lines['eng'] = lines.apply (lambda row: ' '.join(row.clean[1:-1]), axis=1)

In [190]:
# lenght_list=[]
# for l in lines.eng:
#     line = l.split(' ')
#     new_line = []
#     for word in line:
#         if word in all_eng_words:
#             new_line.append(word)
#     lenght_list.append(len(new_line))
# np.max(lenght_list)

In [191]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
# del all_eng_words, all_french_words

In [192]:
input_token_idx = [(word, i) for i, word in enumerate(input_words)]
input_token_index = {}
for (word,i) in input_token_idx:
    input_token_index[word] = i
target_token_idx = [(word, i) for i, word in enumerate(target_words)]
target_token_index = {}
for (word,i) in target_token_idx:
    target_token_index[word] = i

In [193]:
len(lines.fr)*16*num_decoder_tokens

3215271200

In [194]:
encoder_input_data = np.zeros(
    (len(lines.eng), 58), # Need to enter max words manually
    dtype='uint16') # Changed from float 32
decoder_input_data = np.zeros(
    (len(lines.fr), 60),
    dtype='uint16') # Changed from float 32
decoder_target_data = np.zeros(
    (len(lines.fr), 60, num_decoder_tokens),
    dtype='bool') # Changed from float 32


In [195]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.fr)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [196]:
print(encoder_input_data)

[[11261 10835 10197 ...     0     0     0]
 [16086   864  3115 ...     0     0     0]
 [17949 18196 14878 ...     0     0     0]
 ...
 [ 7974 14993 14647 ...     0     0     0]
 [ 7974 14993 14647 ...     0     0     0]
 [14562 16903 10197 ...     0     0     0]]


### Build keras encoder-decoder model

In [197]:
embedding_size = 50

In [198]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

#### Encoder model

In [199]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#### Decoder model

In [200]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


In [201]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 50)     919600      input_19[0][0]                   
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 50)     919700      input_20[0][0]                   
__________________________________________________________________________________________________
lstm_11 (L

#### Fit the model

In [202]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=35,
          validation_split=0.05)

Train on 10378 samples, validate on 547 samples
Epoch 1/35
10378/10378 [==============================] - 69s 7ms/step - loss: 2.6296 - acc: 0.0143 - val_loss: 2.2780 - val_acc: 0.0167
Epoch 2/35
10378/10378 [==============================] - 65s 6ms/step - loss: 2.2890 - acc: 0.0191 - val_loss: 2.2819 - val_acc: 0.0212
Epoch 3/35
10378/10378 [==============================] - 66s 6ms/step - loss: 2.2620 - acc: 0.0219 - val_loss: 2.2798 - val_acc: 0.0206
Epoch 4/35
10378/10378 [==============================] - 66s 6ms/step - loss: 2.2427 - acc: 0.0220 - val_loss: 2.2759 - val_acc: 0.0214
Epoch 5/35
10378/10378 [==============================] - 66s 6ms/step - loss: 2.2242 - acc: 0.0222 - val_loss: 2.2663 - val_acc: 0.0218
Epoch 6/35
10378/10378 [==============================] - 66s 6ms/step - loss: 2.2017 - acc: 0.0230 - val_loss: 2.2550 - val_acc: 0.0233
Epoch 7/35
10378/10378 [==============================] - 66s 6ms/step - loss: 2.1785 - acc: 0.0260 - val_loss: 2.2449 - val_acc: 

In [203]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_11 (Embedding)     (None, None, 50)          919600    
_________________________________________________________________
lstm_11 (LSTM)               [(None, 50), (None, 50),  20200     
Total params: 939,800
Trainable params: 939,800
Non-trainable params: 0
_________________________________________________________________


#### Create sampling model

In [204]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_idx = [(i, char) for char, i in input_token_index.items()]
reverse_input_char_index = {}
for (word,i) in reverse_input_char_idx:
    reverse_input_char_index[word] = i
reverse_target_char_idx = [(i, char) for char, i in target_token_index.items()]
reverse_target_char_index = {}
for (word,i) in reverse_target_char_idx:
    reverse_target_char_index[word] = i

#### Function to generate sequences

In [205]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

#### Look at the some translations

In [206]:
#for seq_index in [14077,20122,40035,40064, 40056, 40068, 40090, 40095, 40100, 40119, 40131, 40136, 40150, 40153]:
import random
comments = list(lines.eng.values)
sample = random.sample(comments,10)
# for seq_index in [500,1000,1500,2000]:
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print('-')
#     print('Input sentence:', lines.eng[seq_index: seq_index + 1])
#     print('Decoded sentence:', decoded_sentence)
#

print("Testing female very toxic")
encoder_data = np.zeros(
    (len(lines.eng), 50), # Need to enter max words manually
    dtype='uint16') # Changed from float 32
for i,input_text in enumerate(sample):
    for t, word in enumerate(input_text.split()):
        encoder_data[i, t] = input_token_index[word]
for i in range(len(sample)):
    print('-')
    print('Input sentence:', sample[i])
    decoded_sentence = decode_sequence(encoder_data[i:i+1])
    print('Decoded sentence:', decoded_sentence)
    
print("\nTesting male very toxic") 
male_vtoxic = very_toxic[very_toxic.male_binary == 1]
male_vtoxic['comment'] = male_vtoxic['comment'].str.translate(str.maketrans('', '', string.punctuation))
male_vtoxic['comment'] = male_vtoxic['comment'].replace('\n',' ', regex=True)
male_vtoxic['comment'] = male_vtoxic['comment'].str.lower()
comments = list(male_vtoxic.comment.values)
sample = random.sample(comments,10)

all_words=set()
lengths = []
for comment in sample:
    for word in comment.split():
        if word not in all_words:
            all_words.add(word)
        lengths.append(len(comment.split()))
max_len = max(lengths)
input_words = sorted(list(all_words))
input_token_idx = [(word, i) for i, word in enumerate(input_words)]
input_token_index = {}
for (word,i) in input_token_idx:
    input_token_index[word] = i
    
encoder_data = np.zeros(
    (len(lines.eng), max_len), # Need to enter max words manually
    dtype='uint16') # Changed from float 32
for i,input_text in enumerate(sample):
    for t, word in enumerate(input_text.split()):
        encoder_data[i, t] = input_token_index[word]
for i in range(len(sample)):
    print('-')
    print('Input sentence:', sample[i])
    decoded_sentence = decode_sequence(encoder_data[i:i+1])
    print('Decoded sentence:', decoded_sentence)

print("\nTesting female healthy")
female_healthy = toxicity[toxicity.toxicity_score == 2.0]
female_healthy = female_healthy[female_healthy.female_binary == 1]
female_healthy['comment'] = female_healthy['comment'].str.translate(str.maketrans('', '', string.punctuation))
female_healthy['comment'] = female_healthy['comment'].replace('\n',' ', regex=True)
female_healthy['comment'] = female_healthy['comment'].str.lower()
comments = list(female_healthy.comment.values)
sample = random.sample(comments,10)

all_words=set()
lengths = []
for comment in sample:
    for word in comment.split():
        if word not in all_words:
            all_words.add(word)
        lengths.append(len(comment.split()))
max_len = max(lengths)
input_words = sorted(list(all_words))
input_token_idx = [(word, i) for i, word in enumerate(input_words)]
input_token_index = {}
for (word,i) in input_token_idx:
    input_token_index[word] = i
    
encoder_data = np.zeros(
    (len(lines.eng), max_len), # Need to enter max words manually
    dtype='uint16') # Changed from float 32
for i,input_text in enumerate(sample):
    for t, word in enumerate(input_text.split()):
        encoder_data[i, t] = input_token_index[word]
for i in range(len(sample)):
    print('-')
    print('Input sentence:', sample[i])
    decoded_sentence = decode_sequence(encoder_data[i:i+1])
    print('Decoded sentence:', decoded_sentence)


Testing female very toxic
-
Input sentence: kiss my ass you stupid nigger jew fuckwit
Decoded sentence:  you are a fucking faggot _END
-
Input sentence: aw youre so sweet too bad your a slutty panda hooker go suck a cock
Decoded sentence:  your a page and i will be a life _END
-
Input sentence: im retired fuckers leave me alone
Decoded sentence:  your a fucking page is a fucking faggot _END
-
Input sentence: therefore you are a pedo you fuck little boys
Decoded sentence:  you are a fucking cunt and die _END
-
Input sentence: warning why did you warn me for what im war with and other jerks warn me for what
Decoded sentence:  i am going to be a idiot to the article i am a fucking
-
Input sentence: there are other valid interpretations whats more interpretations that are backed up by sources bush didnt want to spread panic and if he had out of the room the footage would have been endlessly on television and it wouldnt have done much for national
Decoded sentence:  the article of the artic

/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/nick/anaconda3/envs/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

Decoded sentence:  the article is a fucking page and the fuck off _END
-
Input sentence:    tu mom es a dirty slut    disgusting
Decoded sentence:  is a fucking page _END
-
Input sentence:   congrats for being a total fucking faggot  
Decoded sentence:  is a gay is a fucking _END
-
Input sentence:    what the hell stupid robots i didnt do anything   
Decoded sentence:  the article is a fucking moron _END
-
Input sentence:  mineral county alternative schools 
Decoded sentence:  is a gay _END
-
Input sentence:   go to fuck looser
Decoded sentence:  is a fucking faggot _END
-
Input sentence:    hey ole faggot   hey faggot you are an old man so why dont you retire faggot
Decoded sentence:  i am a fucking article to be a fucking article to be
-
Input sentence:   antipederasty  some catholics accuse noncatholics who believe the catholic church should be held accountable for its sins especially pederasty of being anticatholic  likewise persons who object to aspects of catholic dogma are somet